# Omaha Incidents

## Data Exploration

Let"s explore the data a little bit to see what kind of analysis and visualizations we want to implement.

### Set up environment

Start by installating and importing the necessary packages. 

In [ ]:
# Install packages, if needed
# !pip3 install ipykernel
# !pip3 install --upgrade pandas plotly dash "nbformat>=4.2.0"

In [ ]:
# Import packages
import pandas as pd
import numpy as np
import sqlite3
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

### Load Data

To load the data, we need to connect to the SQLite3 database file and query it for the data we want.

In [ ]:
# Connect to the database
connection = sqlite3.connect("../raw_data/ingress.db")
cursor = connection.cursor()

In [ ]:
# If exists, delete extra header rows
# delete_headers = "DELETE FROM incidents WHERE rb = 'RB Number'"
# cursor.execute(delete_headers)

In [ ]:
# Grab all data
select_all = "SELECT * FROM incidents"
df = pd.read_sql_query(select_all, connection)
df.head()

### Data Cleaning

We will clean up the data before we use: inserting NaN, converting types, etc.

In [ ]:
# Replace empty cells in [lat, lon] with NaN
df = df.replace(r'^\s*$', np.nan, regex=True)
df.head()

In [ ]:
# Convert date col to datetime format
df["date"] = pd.to_datetime(df["date"])
df

### Plotting

Let's test a plot that will show us the top categories of incidents.

In [ ]:
# test plotting by sorting & plotting top 5 crime categories
s = dff.value_counts(subset=["description"])
t = s.nlargest(5)
t.head()
t.plot(kind="bar", title="Top 5 Incident Categories")

### Data Filtering

To reduce the workload in this rest of this notebook, I am filtering just for one description and a range of dates.

If you are doing a lot of analysis, I recommend modifying the query at the beginning to only the pull the data you need instead of filtering after querying.

In [ ]:
# Create a smaller dataframe based on a selected date and description
start_date = "2023-01-01"
end_date = "2023-12-31"
description = "INJURY"

dff = df[(df['date'] > start_date) & (df['date'] < end_date)]
dff = dff.reset_index()
dff = dff[dff.description == description]

dff_grouped = dff.groupby(by="date").count()
dff_grouped = dff_grouped.reset_index()

print(dff.head())
print(dff_grouped.head())

In [ ]:
dff.size

In [ ]:
dff.info()

### Mapping

Let's create a geo map of the crime data.

In [ ]:
fig = px.scatter_mapbox(
    dff,
    lat="lat",
    lon="lon",
    color="description",
    hover_name="description",
    hover_data=["date", "time"],
    title="Incident Count by Coordinates",
    center={"lat": 41.257160, "lon": -95.995102},
    zoom=10
)

# fig.update_layout(showlegend=False)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.update_layout(mapbox_bounds={"west": -180, "east": -50, "south": 20, "north": 90})
fig.show()

In [ ]:
# Optionally, save the figure to an HTML file
# pio.write_html(fig, file="test.html", auto_open=True)

## Wrapping Up

To finish, remember to close your database connections and save any data you need.

In [ ]:
# clean up and close out the database
connection.commit()
connection.close()